Per prima cosa vi ricordo che questo programma e' sperimentale. Nemmeno in alfa o beta status. E' una prova di concetto sperimentale e il suo uso potrebbe costarvi soldi o, peggio farvi perdere dati. 

Ad ogni modo il (c) e' di Giovambattista Vieri  e, la licenza e' quella che volete (WTF o mit o come vi pare)

Per prima cosa e' necessario fare il download del file dati. https://www.kaggle.com/dansbecker/5-celebrity-faces-dataset/downloads/5-celebrity-faces-dataset.zip/3

Lo troverete sul vostro computer. Si chiama 5-celebrity-faces-dataset.zip e, e' necessario scompattarlo. Al suo interno troverete un file zip che dovrete poi caricare. Si chiama data.zip. 

Ora procediamo. 


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:


import os 
os.mkdir('data')



In [0]:
os.mkdir('models')

In [0]:
!cp data.zip data
!cd data
!unzip data.zip

In [0]:

!ls -l data 
!mv train data
!mv val data

Presumibilmente ora i dati sono stati caricati. 
Passiamo all'ambiente


In [0]:
!ls -la data

In [0]:
!pip install numpy matplotlib
!pip install torchvision

finalmente abbiam finito i convenevoli 

passiamo al codice: 


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim


In [0]:
num_of_cat=5

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}


In [0]:
image_datasets = {
    'train':
    datasets.ImageFolder('data/train', data_transforms['train']),
    'validation':
    datasets.ImageFolder('data/val', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=15,
                                shuffle=True, num_workers=4),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=15,
                                shuffle=False, num_workers=4)
}



In [0]:
device = torch.device("cuda") # I want to be able to repeat ... 
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, num_of_cat)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())


In [0]:
def train_model(model, criterion, optimizer, num_epochs=6):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.float() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss.item(),
                                                        epoch_acc.item()))
    return model


In [0]:
model_trained = train_model(model, criterion, optimizer, num_epochs=6)



In [0]:
torch.save(model_trained.state_dict(), 'models/weights.h5')

ora il modello e' allenato, e, salvato... 

In [0]:

validation_img_paths = ["data/val/elton_john/httpcdnlyricssongonlyricsnetwpcontentuploadsEltonJohnDiscographyCDreleasesjpg.jpg",
                        "data/val/madonna/httpecximagesamazoncomimagesIfmaBKWLACULSRjpg.jpg",
                        "data/val/madonna/httpcdncdnjustjaredcomwpcontentuploadsheadlinesmadonnatalksparisattackstearsjpg.jpg",
                        "data/val/jerry_seinfeld/httpblognjcomentertainmentimpactcelebritiesmediumjerrybjpg.jpg",
                        "data/val/mindy_kaling/httpcdnpastemagazinecomwwwarticlesmindyprojectjpg.jpg",
                        "data/val/ben_afflek/httpabsolumentgratuitfreefrimagesbenaffleckjpg.jpg"]
img_list = [Image.open(img_path) for img_path in validation_img_paths]


In [0]:
img_list = [Image.open(img_path) for img_path in validation_img_paths]


In [0]:
validation_batch = torch.stack([data_transforms['validation'](img).to(device) for img in img_list])

pred_logits_tensor = model(validation_batch)
pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()


predizione effettuata...
VEDIAMO I RISULTATI

In [0]:
fig, axs = plt.subplots(1, len(img_list), figsize=(15, 5))
#fig, axs = plt.subplots(1, len(img_list),figsize=(20,5))


for i, img in enumerate(img_list):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}% BA, {:.0f}% EJ,\n {:.0f}% JS, {:.0f}% M, {:.0f}% MK".format(100*pred_probs[i,0], 100*pred_probs[i,1],100*pred_probs[i,2],100*pred_probs[i,3],100*pred_probs[i,4]))
    ax.imshow(img)
fig.show()
